# Asignación 1: Implementación y Análisis de KNN desde Cero

## Objetivos

- Familiarizarse con conceptos y habilidades básicas de programación en Python aplicadas a ciencia de datos.
- Comprender el funcionamiento del algoritmo K-Nearest Neighbors (KNN) a través de su implementación desde cero.
- Cargar, explorar y preparar el dataset Pima Indian Diabetes para tareas de clasificación.
- Implementar funciones fundamentales como la distancia euclidiana y la predicción de clases usando KNN.
- Comparar la implementación propia de KNN con la versión de scikit-learn, analizando similitudes y diferencias en los resultados.
- Analizar el impacto del parámetro **k** en el desempeño del clasificador y discutir los hallazgos.

## Preámbulo: **Dataset Pima Indian Diabetes**

El **Pima Indian Diabetes Dataset** es un conjunto de datos clásico en Machine Learning y estadística, utilizado principalmente para problemas de clasificación binaria. Contiene información médica de mujeres de origen Pima (una población indígena de América del Norte) mayores de 21 años. Cada registro incluye variables como número de embarazos, concentración de glucosa en plasma, presión arterial, grosor del pliegue cutáneo, niveles de insulina, índice de masa corporal (IMC), función hereditaria de la diabetes y edad. El objetivo es predecir si una persona tiene o no diabetes (variable objetivo binaria).

Un aspecto importante de este dataset es la presencia de valores faltantes, especialmente en variables como glucosa, presión arterial, grosor del pliegue cutáneo e insulina, donde valores igual a cero suelen indicar datos ausentes. Para esta asignación, se utilizará una versión limpia del dataset en la que se han eliminado los registros con valores faltantes, permitiendo así un análisis más directo y sin la necesidad de imputación de datos.

## Paso 1: Cargar y explorar el dataset

**Instrucciones:**
- Descarga el dataset desde el repositorio de GitHub del curso. El archivo se encuentra en `datasets/pima_indian_diabetes_dataset/cleaned_dataset.csv`.
- Carga el dataset utilizando pandas.
- Muestra las primeras filas (`df.head()`) del dataset.
- Imprime la cantidad total de filas y columnas del dataset.

In [124]:
import pandas as pd

# Cargar el dataset
df = pd.read_csv('../datasets/pima_indian_diabetes_dataset/cleaned_dataset.csv')

# Mostrar las primeras filas
print(df.head())

# Imprimir la cantidad de filas y columnas
print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")


   Pregnancies  Glucose  Blood Pressure  Skin Thickness  Insulin   BMI  \
0            0      129             110              46      130  67.1   
1            0      180              78              63       14  59.4   
2            3      123             100              35      240  57.3   
3            1       88              30              42       99  55.0   
4            0      162              76              56      100  53.2   

   Diabetes Pedigree Function  Age  Outcome  
0                       0.319   26        1  
1                       2.420   25        1  
2                       0.880   22        0  
3                       0.496   26        1  
4                       0.759   25        1  
Filas: 392, Columnas: 9


## Paso 2: Crear función para cargar y dividir el dataset

**Instrucciones:**
- Implementa una función en Python que:
  - Cargue el dataset limpio desde la ruta especificada.
  - Seleccione las primeras 10 muestras como conjunto de entrenamiento.
  - Seleccione las siguientes 10 muestras como conjunto de prueba.
  - Devuelva por separado: `X_train`, `y_train`, `X_test`, `y_test`.

In [125]:
import numpy as np

# Tu código aquí
def cargar_y_dividir_dataset(df):
    # Seleccionar las primeras 10 muestras para entrenamiento
    train = df.iloc[:10]
    # Seleccionar las siguientes 10 muestras para prueba
    test = df.iloc[10:20]
    # Separar características y etiquetas
    X_train = train.drop('Outcome', axis=1).values
    y_train = train['Outcome'].values
    X_test = test.drop('Outcome', axis=1).values
    y_test = test['Outcome'].values
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = cargar_y_dividir_dataset(df)



## Paso 3: Implementar la función de distancia euclidiana

**Instrucciones:**
- Escribe una función en Python que reciba dos vectores y calcule la distancia euclidiana entre ellos.
- Utiliza la siguiente fórmula matemática para la distancia euclidiana entre dos vectores $x$ y $y$ de $n$ dimensiones:

$$
d(x, y) = \sqrt{\sum_{i=1}^{n} (x_i - y_i)^2}
$$

- Prueba tu función con los siguientes dos ejemplos (cada vector corresponde a una fila del dataset):

| Embarazos | Glucosa | Presión Arterial | Grosor Piel | Insulina | IMC  | Función Hereditaria | Edad | Resultado |
|-----------|---------|------------------|-------------|----------|------|---------------------|------|-----------|
|     1     |   106   |        70        |      28     |   135    | 34.2 |        0.142        |  22  |     0     |
|     2     |   102   |        86        |      36     |   120    | 45.5 |        0.127        |  23  |     1     |

- Calcula la distancia euclidiana a mano y luego verifica que el resultado de tu función sea el mismo.
- La función debe imprimir el resultado del cálculo de la distancia euclidiana con los datos presentados.

In [126]:
# Tu código aquí

def euclidean_distance(x, y):
    return np.sqrt(np.sum((x - y) ** 2))

# Ejemplo con los dos vectores dados en el enunciado:
x = np.array([1, 106, 70, 28, 135, 34.2, 0.142, 22])
y = np.array([2, 102, 86, 36, 120, 45.5, 0.127, 23])

dist = euclidean_distance(x, y)
print(f"Distancia euclidiana entre los dos vectores de ejemplo: {dist:.4f}")


Distancia euclidiana entre los dos vectores de ejemplo: 26.2810


## Paso 4: Implementar un clasificador KNN básico

**Instrucciones:**
- Escribe una función que, dado un punto de prueba, calcule la distancia a todos los puntos de entrenamiento utilizando tu función de distancia euclidiana.
- Selecciona los **k = 3** vecinos más cercanos y predice la clase mayoritaria entre ellos.
- Aplica tu función a las 10 muestras de prueba obtenidas previamente, utilizando las 10 muestras de entrenamiento como referencia.
- El script debe imprimir una tabla comparando el valor real de `Resultado` de cada muestra de prueba con el valor predicho por tu algoritmo.
- Considere que las tablas se pueden codificar con un formato similar al que se muestra en el siguiente código:

In [127]:
# Ejemplo de tabla simple usando print y formato alineado
print("{:<8} {:<13} {:<8}".format("Muestra", "Resultado_real", "Predicho"))
print("{:<8} {:<13} {:<8}".format(1, 0, 0))
print("{:<8} {:<13} {:<8}".format(2, 1, 1))
print("{:<8} {:<13} {:<8}".format(3, 0, 0))
print("{:<8} {:<13} {:<8}".format(4, 1, 1))
print("{:<8} {:<13} {:<8}".format(5, 0, 1))

Muestra  Resultado_real Predicho
1        0             0       
2        1             1       
3        0             0       
4        1             1       
5        0             1       


In [128]:

def knn_predict(X_train, y_train, x_test, k=3):
    # Calcular distancias euclidianas entre x_test y todos los puntos de entrenamiento
    distances = []
    for x_train in X_train:
        dist = euclidean_distance(x_test, x_train)
        distances.append(dist)     
   
    # Obtener los índices de los k vecinos más cercanos
    neighbors_idx = np.argsort(distances)[:k]    
    
    # Obtener las etiquetas de los vecinos más cercanos
    neighbor_labels = y_train[neighbors_idx]
   
    # Contar la cantidad de ocurrencias de cada clase
    counts = np.bincount(neighbor_labels)
    # Seleccionar la clase mayoritaria
    predicted_class = np.argmax(counts)
    return predicted_class

# Predecir para todas las muestras de prueba
y_pred = []
for i, x_test in enumerate(X_test):
    pred = knn_predict(X_train, y_train, x_test, k=3)
    y_pred.append(pred)

# Imprimir tabla comparativa
print("{:<8} {:<13} {:<8}".format("Muestra", "Resultado_real", "Predicho"))
for i, (real, pred) in enumerate(zip(y_test, y_pred), 1):
    print("{:<8} {:<13} {:<8}".format(i, real, pred))


Muestra  Resultado_real Predicho
1        0             0       
2        0             0       
3        1             0       
4        0             1       
5        1             1       
6        1             0       
7        1             0       
8        1             1       
9        1             0       
10       0             1       


## Paso 5: Comparar con scikit-learn

**Instrucciones:**
- Utiliza `KNeighborsClassifier` de scikit-learn para entrenar y predecir sobre el mismo subconjunto de datos. Asegúrate de definir los hiperparámetros: `k=3`, distancia euclidiana y método de búsqueda fuerza bruta (`algorithm='brute'`).
- Compara los resultados de tu implementación con los obtenidos por scikit-learn.
- El script debe mostrar una tabla que compare el valor real de `Resultado` de cada muestra de prueba, el valor predicho por tu algoritmo y el valor predicho por scikit-learn.

In [129]:
from sklearn.neighbors import KNeighborsClassifier

# Tu código aquí

# Entrenar el clasificador de scikit-learn
knn_sklearn = KNeighborsClassifier(n_neighbors=3, metric='euclidean', algorithm='brute')
knn_sklearn.fit(X_train, y_train)
y_pred_sklearn = knn_sklearn.predict(X_test)

# Imprimir tabla comparativa
print("{:<8} {:<15} {:<10} {:<10}".format("Muestra", "Resultado_real", "Propio", "sklearn"))
for idx, (real, propio, skl) in enumerate(zip(y_test, y_pred, y_pred_sklearn), 1):
    print("{:<8} {:<15} {:<10} {:<10}".format(idx, real, propio, skl))


Muestra  Resultado_real  Propio     sklearn   
1        0               0          0         
2        0               0          0         
3        1               0          0         
4        0               1          1         
5        1               1          1         
6        1               0          0         
7        1               0          0         
8        1               1          1         
9        1               0          0         
10       0               1          1         


## Paso 6: Normalización de los datos

Hasta ahora, se han utilizado los datos crudos directamente desde el dataset. Sin embargo, en Machine Learning es una buena práctica normalizar o escalar los datos antes de aplicar algoritmos basados en distancias, como KNN. La normalización ayuda a que todas las variables tengan el mismo rango y evita que aquellas con valores numéricos grandes dominen el cálculo de distancias.

**Instrucciones:**
- Implementa una nueva función de carga de datos, similar a la del Paso 2, que:
  - Cargue el dataset limpio desde la ruta especificada.
  - Seleccione las primeras 10 muestras como conjunto de entrenamiento y las siguientes 10 como conjunto de prueba.
  - Aplique un escalado Min-Max (`MinMaxScaler` de scikit-learn) a las 20 muestras seleccionadas, considerando el mínimo y el máximo de ambos conjuntos.
  - Devuelva los conjuntos `X_train`, `y_train`, `X_test`, `y_test` ya normalizados.
- Utiliza estos datos normalizados para volver a comparar el desempeño de tu implementación de KNN y la de scikit-learn, como en el Paso 5.
- Presenta los resultados en una tabla comparativa.

In [132]:
from sklearn.preprocessing import MinMaxScaler
from IPython.display import display

# Tu código aquí

def cargar_y_dividir_y_normalizar(df):
    # Seleccionar las primeras 20 muestras
    subset = df.iloc[:20]
    # Separar características y etiquetas
    X = subset.drop('Outcome', axis=1).values
    y = subset['Outcome'].values
    # Normalizar usando MinMaxScaler
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    # Dividir en train y test
    X_train_norm = X_scaled[:10]
    y_train_norm = y[:10]
    X_test_norm = X_scaled[10:20]
    y_test_norm = y[10:20]
    return X_train_norm, y_train_norm, X_test_norm, y_test_norm


# Obtener los datos normalizados
X_train_norm, y_train_norm, X_test_norm, y_test_norm = cargar_y_dividir_y_normalizar(df)

    # Predecir con implementación propia
y_pred_norm = []
for x_test in X_test_norm:
        pred = knn_predict(X_train_norm, y_train_norm, x_test, k=3)
        y_pred_norm.append(pred)

# Predecir con scikit-learn
knn_sklearn_norm = KNeighborsClassifier(n_neighbors=3, metric='euclidean', algorithm='brute')
knn_sklearn_norm.fit(X_train_norm, y_train_norm)
y_pred_sklearn_norm = knn_sklearn_norm.predict(X_test_norm)

# Imprimir tabla comparativa
print("{:<8} {:<15} {:<10} {:<10}".format("Muestra", "Resultado_real", "Propio", "sklearn"))
for idx, (real, propio, skl) in enumerate(zip(y_test_norm, y_pred_norm, y_pred_sklearn_norm), 1):
        print("{:<8} {:<15} {:<10} {:<10}".format(idx, real, propio, skl))


Muestra  Resultado_real  Propio     sklearn   
1        0               1          1         
2        0               1          1         
3        1               0          0         
4        0               0          0         
5        1               0          0         
6        1               0          0         
7        1               0          0         
8        1               0          0         
9        1               0          0         
10       0               0          0         


## Paso 7: Analizar el impacto de k

**Instrucciones:**
- Evalúa el desempeño de tu implementación personalizada para distintos valores de **k** (por ejemplo: 1, 3, 5, 7, 9).
- Utiliza los datos normalizados.
- Para cada valor de k, predice el `Resultado` de las muestras de prueba.
- Construye una tabla que muestre, para cada muestra de prueba, el número de muestra, el valor real de `Resultado` y los valores predichos por tu algoritmo para cada valor de k. Por ejemplo:

| Muestra | Resultado real | k=1 | k=3 | k=5 | k=7 | k=9 |
|---------|----------------|-----|-----|-----|-----|-----|
|    1    |       0        |  0  |  0  |  1  |  0  |  0  |
|    2    |       1        |  1  |  1  |  0  |  1  |  1  |
|    3    |       0        |  0  |  0  |  0  |  0  |  1  |
|    4    |       1        |  1  |  0  |  1  |  1  |  1  |
|    5    |       0        |  0  |  1  |  0  |  0  |  0  |
|    6    |       1        |  1  |  1  |  1  |  1  |  1  |
|    7    |       0        |  0  |  0  |  0  |  1  |  0  |
|    8    |       1        |  1  |  1  |  1  |  1  |  1  |
|    9    |       0        |  0  |  0  |  1  |  0  |  0  |
|   10    |       1        |  1  |  1  |  1  |  1  |  1  |

In [ ]:
# Análisis del impacto del parámetro k en el clasificador KNN

# Definir los valores de k a evaluar
ks = [1, 3, 5, 7, 9]
predicciones_por_k = []

# Para cada valor de k, realizar predicciones en todas las muestras de prueba
for k in ks:
    y_pred_k = []
    # Predecir cada muestra de prueba con el valor actual de k
    for x_test in X_test_norm:
        pred = knn_predict(X_train_norm, y_train_norm, x_test, k=k)
        y_pred_k.append(pred)
    # Almacenar las predicciones para este valor de k
    predicciones_por_k.append(y_pred_k)

# Crear tabla comparativa mostrando el impacto de diferentes valores de k
print("{:<8} {:<15}".format("Muestra", "Resultado real"), end="")
for k in ks:
    print("k={:<4}".format(k), end="")
print()

# Imprimir resultados para cada muestra de prueba
for idx in range(len(y_test_norm)):
    print("{:<8} {:<15}".format(idx+1, y_test_norm[idx]), end="")
    # Mostrar la predicción de cada valor de k para esta muestra
    for y_pred_k in predicciones_por_k:
        print("{:<6}".format(y_pred_k[idx]), end="")
    print()

Muestra  Resultado real k=1   k=3   k=5   k=7   k=9   
1        0              0     1     0     0     0     
2        0              0     1     0     0     0     
3        1              0     0     0     0     0     
4        0              0     0     0     0     0     
5        1              1     0     0     0     0     
6        1              1     0     0     0     0     
7        1              1     0     0     0     0     
8        1              1     0     0     0     0     
9        1              0     0     0     0     0     
10       0              1     0     0     0     0     


# Rúbrica de Evaluación

| Paso                                                         | Puntos |
|--------------------------------------------------------------|--------|
| 1. Cargar y explorar el dataset                              |   10   |
| 2. Crear función para cargar y dividir el dataset            |   10   |
| 3. Implementar la función de distancia euclidiana            |   10   |
| 4. Implementar un clasificador KNN básico                    |   10   |
| 5. Comparar con scikit-learn                                 |   20   |
| 6. Normalización y comparación con KNN y scikit-learn        |   20   |
| 7. Analizar el impacto de k                                  |   20   |
| **Total**                                                    | **100**|